In [108]:
!pip install lightgbm

  Using cached lightgbm-4.6.0-py3-none-win_amd64.whl.metadata (17 kB)
Using cached lightgbm-4.6.0-py3-none-win_amd64.whl (1.5 MB)


In [83]:
import pandas as pd
import numpy as np
import glob
import os
from datetime import datetime
from collections import defaultdict
import re
from pathlib import Path
from io import StringIO

In [84]:
clients = pd.read_csv('E:\Alex\Github\Decentraphone\data\case 1\clients.csv')
clients.describe()

,client_code,age,avg_monthly_balance_KZT
count,60.000000,60.000000,6.000000e+01
mean,30.500000,37.516667,1.094079e+06
std,17.464249,9.145698,1.417719e+06
min,1.000000,20.000000,4.663000e+04
25%,15.750000,30.000000,8.169800e+04
50%,30.500000,36.500000,1.483350e+05
75%,45.250000,44.000000,1.433970e+06
max,60.000000,58.000000,5.818675e+06


In [85]:
c1 = pd.read_csv('E:\Alex\Github\Decentraphone\data\case 1\client_1_transactions_3m.csv')

In [ ]:

def unify_and_save_clients(src_path: str, dst_path: str, n_clients: int = 60):
    os.makedirs(dst_path, exist_ok=True)

    for i in range(1, n_clients + 1):
        tr_file = os.path.join(src_path, f"client_{i}_transactions_3m.csv")
        tf_file = os.path.join(src_path, f"client_{i}_transfers_3m.csv")
        out_file = os.path.join(dst_path, f"client_{i}_merged.csv")

        if os.path.exists(tr_file) and os.path.exists(tf_file):
            df_tr = pd.read_csv(tr_file, parse_dates=["date"])
            df_tf = pd.read_csv(tf_file, parse_dates=["date"])

            # --- нормализация ---
            df_tr["type"] = "purchase"
            df_tr["direction"] = "out"

            df_tf["category"] = df_tf["type"]

            cols = [
                "client_code","name","product","status","city",
                "date","type","direction","category","amount","currency"
            ]
            df_tr = df_tr[cols]
            df_tf = df_tf[cols]

            merged_df = pd.concat([df_tr, df_tf], ignore_index=True)
            merged_df.to_csv(out_file, index=False, encoding="utf-8-sig")
            print(f"[OK] Клиент {i}: сохранено в {out_file}")
        else:
            print(f"[SKIP] Нет файлов для клиента {i}")


In [87]:
src = r"E:\Alex\Github\Decentraphone\data\case 1"
dst = r"E:\Alex\Github\Decentraphone\data\processed_data"
unify_and_save_clients(src, dst, n_clients=60)


[OK] Клиент 1: сохранено в E:\Alex\Github\Decentraphone\data\processed_data\client_1_merged.csv
[OK] Клиент 2: сохранено в E:\Alex\Github\Decentraphone\data\processed_data\client_2_merged.csv
[OK] Клиент 3: сохранено в E:\Alex\Github\Decentraphone\data\processed_data\client_3_merged.csv
[OK] Клиент 4: сохранено в E:\Alex\Github\Decentraphone\data\processed_data\client_4_merged.csv
[OK] Клиент 5: сохранено в E:\Alex\Github\Decentraphone\data\processed_data\client_5_merged.csv
[OK] Клиент 6: сохранено в E:\Alex\Github\Decentraphone\data\processed_data\client_6_merged.csv
[OK] Клиент 7: сохранено в E:\Alex\Github\Decentraphone\data\processed_data\client_7_merged.csv
[OK] Клиент 8: сохранено в E:\Alex\Github\Decentraphone\data\processed_data\client_8_merged.csv
[OK] Клиент 9: сохранено в E:\Alex\Github\Decentraphone\data\processed_data\client_9_merged.csv
[OK] Клиент 10: сохранено в E:\Alex\Github\Decentraphone\data\processed_data\client_10_merged.csv
[OK] Клиент 11: сохранено в E:\Alex\Gi

In [88]:
a = pd.read_csv(r"E:\Alex\Github\Decentraphone\data\processed_data\client_1_merged.csv")
a.head()

,client_code,name,product,status,city,date,type,direction,category,amount,currency
0,1,Айгерим,Карта для путешествий,зп,Алматы,2025-06-01 09:10:36,purchase,out,Такси,6424.48,KZT
1,1,Айгерим,Карта для путешествий,зп,Алматы,2025-06-01 09:15:49,purchase,out,Такси,2643.90,KZT
2,1,Айгерим,Карта для путешествий,зп,Алматы,2025-06-01 13:10:49,purchase,out,Смотрим дома,4716.59,KZT
3,1,Айгерим,Карта для путешествий,зп,Алматы,2025-06-01 15:40:10,purchase,out,Такси,3669.43,KZT
4,1,Айгерим,Карта для путешествий,зп,Алматы,2025-06-01 17:40:30,purchase,out,Играем дома,5095.03,KZT


In [89]:
clients_path_pattern = 'E:\Alex\Github\Decentraphone\data\case 1\clients.csv'       
tx_pattern = 'E:\Alex\Github\Decentraphone\data\processed_data\client_*_merged.csv'         
output_path = 'E:\Alex\Github\Decentraphone\output\enefits_per_client.csv'

In [90]:
clients_files = glob.glob(clients_path_pattern, recursive=True)
if len(clients_files) == 0:
    print("Warning: не найден")
else:
    clients = pd.read_csv(clients_files[0])

In [91]:
tx_files = glob.glob(tx_pattern, recursive=True)
if len(tx_files) == 0:
    print("Warning: не найдены")
else:

    dfs = []
    for f in tx_files:
        try:
            d = pd.read_csv(f, parse_dates=['date'])
            dfs.append(d)
            print(f"Loaded {f} ({d.shape[0]} rows)")
        except Exception as e:
            print(f"Failed to read {f}: {e}")
    if len(dfs) == 0:
        raise RuntimeError("Не удалось загрузить ни один файл транзакций.")
    tx = pd.concat(dfs, ignore_index=True)
    print(f"Total transactions loaded: {tx.shape[0]} rows from {len(dfs)} files")


Loaded E:\Alex\Github\Decentraphone\data\processed_data\client_10_merged.csv (600 rows)
Loaded E:\Alex\Github\Decentraphone\data\processed_data\client_11_merged.csv (600 rows)
Loaded E:\Alex\Github\Decentraphone\data\processed_data\client_12_merged.csv (600 rows)
Loaded E:\Alex\Github\Decentraphone\data\processed_data\client_13_merged.csv (600 rows)
Loaded E:\Alex\Github\Decentraphone\data\processed_data\client_14_merged.csv (600 rows)
Loaded E:\Alex\Github\Decentraphone\data\processed_data\client_15_merged.csv (600 rows)
Loaded E:\Alex\Github\Decentraphone\data\processed_data\client_16_merged.csv (600 rows)
Loaded E:\Alex\Github\Decentraphone\data\processed_data\client_17_merged.csv (600 rows)
Loaded E:\Alex\Github\Decentraphone\data\processed_data\client_18_merged.csv (600 rows)
Loaded E:\Alex\Github\Decentraphone\data\processed_data\client_19_merged.csv (600 rows)
Loaded E:\Alex\Github\Decentraphone\data\processed_data\client_1_merged.csv (600 rows)
Loaded E:\Alex\Github\Decentrapho

In [92]:
tx['amount'] = pd.to_numeric(tx['amount'], errors='coerce').fillna(0.0)
tx['abs_amount'] = tx['amount'].abs()
if 'date' in tx.columns:
    tx['date'] = pd.to_datetime(tx['date'], errors='coerce')
else:
    tx['date'] = pd.NaT

conv_rates = {'KZT': 1.0, 'USD': 460.0, 'EUR': 500.0}  
def convert_to_kzt(row):
    cur = str(row.get('currency','KZT'))
    rate = conv_rates.get(cur, np.nan)
    if np.isnan(rate):
        return row['amount']
    return row['amount'] * rate

tx['amount_kzt'] = tx.apply(convert_to_kzt, axis=1)

In [93]:
travel_categories = set(['Такси','Авиабилеты','Отели','Аренда авто','Поезд','Поезда','Самолет','Ж/д','Авиабилет','Такси/Рейс','Авиабилеты'])
premium_categories = set(['Ювелирные изделия','Парфюмерия','Рестораны','Ресторан','Кафе'])
credit_online_categories = set(['Игры','Доставка','Кино','Онлайн','Подписка','Streaming','Delivery'])

In [94]:
def build_features_for_client(g_tx, client_row):
    if g_tx.shape[0] > 1 and g_tx['date'].notna().any():
        period_days = max((g_tx['date'].max() - g_tx['date'].min()).days, 0)
    else:
        period_days = 0
    period_months = max(1.0, period_days / 30.0)
    
    total_spend = g_tx.loc[g_tx['direction']=='out','amount_kzt'].sum()
    total_inflow = g_tx.loc[g_tx['direction']=='in','amount_kzt'].sum()
    avg_monthly_outflow = total_spend / period_months
    avg_monthly_inflow = total_inflow / period_months
    tx_count = g_tx.shape[0]
    tx_count_out = g_tx.loc[g_tx['direction']=='out'].shape[0]
    freq_card_out = tx_count_out / period_months
    cat_sum = g_tx.loc[g_tx['direction']=='out'].groupby('category')['amount_kzt'].sum().sort_values(ascending=False)
    top3 = cat_sum.index.tolist()[:3]
    top1 = top3[0] if len(top3)>0 else None
    share_top1 = (cat_sum.iloc[0] / total_spend) if (total_spend>0 and len(cat_sum)>0) else 0.0
    
    travel_spend = g_tx.loc[(g_tx['direction']=='out') & (g_tx['category'].isin(travel_categories)),'amount_kzt'].sum()
    travel_tx_count = g_tx.loc[(g_tx['direction']=='out') & (g_tx['category'].isin(travel_categories))].shape[0]
    avg_travel_ticket = (g_tx.loc[(g_tx['direction']=='out') & (g_tx['category'].isin(travel_categories)),'amount_kzt'].mean()
                         if travel_tx_count>0 else 0.0)
    travel_share = travel_spend / total_spend if total_spend>0 else 0.0
    
    premium_spend = g_tx.loc[(g_tx['direction']=='out') & (g_tx['category'].isin(premium_categories)),'amount_kzt'].sum()
    premium_share = premium_spend / total_spend if total_spend>0 else 0.0
    
    online_tx_count = g_tx.loc[(g_tx['direction']=='out') & (g_tx['category'].isin(credit_online_categories))].shape[0]
    online_freq = online_tx_count / period_months
    
    foreign_sum = g_tx.loc[g_tx['currency'].isin(['USD','EUR']),'amount_kzt'].abs().sum()
    total_abs = g_tx['amount_kzt'].abs().sum()
    usd_eur_share = foreign_sum / total_abs if total_abs>0 else 0.0
    fx_tx_count = g_tx.loc[g_tx['currency'].isin(['USD','EUR'])].shape[0]
    fx_freq = fx_tx_count / period_months
    
    if g_tx['date'].notna().any():
        g_tx = g_tx.copy()
        g_tx['month'] = g_tx['date'].dt.to_period('M')
        monthly_net = g_tx.groupby('month').apply(lambda df: df.loc[df['direction']=='in','amount_kzt'].sum() - df.loc[df['direction']=='out','amount_kzt'].sum())
        vol_cashflow = monthly_net.std() if len(monthly_net)>1 else 0.0
    else:
        vol_cashflow = 0.0
    
    small_tx_count = g_tx.loc[g_tx['abs_amount'] < 1000,'amount_kzt'].shape[0]
    small_tx_freq = small_tx_count / period_months
    
    avg_balance = float(client_row.get('avg_monthly_balance_KZT', 0.0)) if client_row is not None else 0.0
    neg_balance_risk = max(0.0, (avg_monthly_outflow - avg_balance) / (avg_balance + 1e-9))
    
    return {
        'period_months': period_months,
        'total_spend': total_spend,
        'total_inflow': total_inflow,
        'avg_monthly_outflow': avg_monthly_outflow,
        'avg_monthly_inflow': avg_monthly_inflow,
        'tx_count': tx_count,
        'tx_count_out': tx_count_out,
        'freq_card_out': freq_card_out,
        'top1_category': top1,
        'share_top1': share_top1,
        'travel_spend': travel_spend,
        'travel_tx_count': travel_tx_count,
        'avg_travel_ticket': avg_travel_ticket,
        'travel_share': travel_share,
        'premium_spend': premium_spend,
        'premium_share': premium_share,
        'online_freq': online_freq,
        'usd_eur_share': usd_eur_share,
        'fx_freq': fx_freq,
        'vol_cashflow': vol_cashflow,
        'small_tx_freq': small_tx_freq,
        'avg_balance': avg_balance,
        'neg_balance_risk': neg_balance_risk
    }


In [95]:
client_rows = []
for _, client in clients.iterrows():
    cid = client['client_code']
    g = tx.loc[tx['client_code']==cid].copy()
    feats = build_features_for_client(g, client)
    row = {**{'client_code': cid, 'name': client.get('name','')}, **feats, **client.to_dict()}
    client_rows.append(row)

features = pd.DataFrame(client_rows).drop_duplicates(subset=['client_code']).reset_index(drop=True)

C:\Users\User\AppData\Local\Temp\ipykernel_25156\2689962368.py:41: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  monthly_net = g_tx.groupby('month').apply(lambda df: df.loc[df['direction']=='in','amount_kzt'].sum() - df.loc[df['direction']=='out','amount_kzt'].sum())
C:\Users\User\AppData\Local\Temp\ipykernel_25156\2689962368.py:41: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  monthly_net = g_tx.groupby('month').app

In [96]:
def normalize_min_p90(s):
    lo = s.min()
    hi = s.quantile(0.90)
    if hi <= lo:
        return ((s - lo) / (hi - lo + 1e-9)).clip(0,1)
    return ((s - lo) / (hi - lo)).clip(0,1)

norm_cols = ['avg_monthly_outflow','tx_count_out','travel_share','travel_tx_count','avg_travel_ticket',
             'premium_share','online_freq','usd_eur_share','fx_freq','vol_cashflow','small_tx_freq','avg_balance','neg_balance_risk']

for col in norm_cols:
    if col not in features.columns:
        features[col] = 0.0
    features['n_' + col] = normalize_min_p90(features[col])

def deposit_tier(balance):
    if balance >= 6_000_000:
        return 1.0
    if balance >= 1_000_000:
        return 0.5
    return 0.0
features['n_deposit_tier'] = features['avg_balance'].apply(deposit_tier)
features['n_share_in_top3'] = features['share_top1'].clip(0,1) 


In [97]:
features['benefit_travel'] = 100 * (
    0.55 * features['n_travel_share'] +
    0.20 * features['n_travel_tx_count'] +
    0.15 * features['n_avg_travel_ticket'] +
    0.10 * 0.0
)
features['benefit_premium'] = 100 * (
    0.40 * features['n_deposit_tier'] +
    0.30 * features['n_avg_monthly_outflow'] +
    0.15 * features['n_tx_count_out'] +
    0.15 * features['n_premium_share']
)
features['benefit_credit'] = 100 * (
    0.45 * features['n_share_in_top3'] +
    0.25 * features['n_online_freq'] +
    0.30 * features['n_neg_balance_risk']
)
features['benefit_fx'] = 100 * (
    0.50 * features['n_fx_freq'] +
    0.35 * features['n_usd_eur_share'] +
    0.15 * 0.0
)
features['benefit_cashloan'] = 100 * (
    0.45 * features['n_neg_balance_risk'] +
    0.35 * features['n_vol_cashflow'] +
    0.20 * (1.0 - features['n_deposit_tier'])
)
features['benefit_saving'] = 100 * (
    0.70 * features['n_avg_balance'] +
    0.30 * (1.0 - features['n_neg_balance_risk'])
)
features['benefit_multidep'] = 100 * (
    0.50 * features['n_avg_balance'] +
    0.30 * features['n_fx_freq'] +
    0.20 * features['n_usd_eur_share']
)
features['benefit_accumulative'] = 100 * (
    0.60 * features['n_avg_balance'] +
    0.40 * (1.0 - features['n_neg_balance_risk'])
)
features['benefit_invest'] = 100 * (
    0.50 * features['n_avg_balance'] +
    0.30 * features['n_small_tx_freq'] +
    0.20 * 0.0
)
features['benefit_gold'] = 100 * (
    0.60 * features['n_avg_balance'] +
    0.30 * (1.0 - features['n_neg_balance_risk']) +
    0.10 * 0.0
)

In [98]:
benefit_cols = [c for c in features.columns if c.startswith('benefit_')]
features['best_product'] = features[benefit_cols].idxmax(axis=1).str.replace('benefit_','')

def pick_with_tiebreaker(row):
    vals = row[benefit_cols].sort_values(ascending=False)
    best = vals.index[0].replace('benefit_','')
    second = vals.index[1].replace('benefit_','') if len(vals)>1 else None
    best_val = vals.iloc[0]
    second_val = vals.iloc[1] if len(vals)>1 else 0.0
    if best_val > 0 and (best_val - second_val) < 0.05 * best_val:
        top1 = row.get('top1_category', None)
        if pd.notna(top1):
            if top1 in travel_categories:
                return 'travel'
            if top1 in premium_categories:
                return 'premium'
            if top1 in credit_online_categories:
                return 'credit'
    return best

features['best_product_tiebreak'] = features.apply(pick_with_tiebreaker, axis=1)

In [99]:
if 'avg_balance' not in features.columns:
    if 'avg_monthly_balance_KZT' in clients.columns:
        \
        tmp = clients[['client_code','avg_monthly_balance_KZT']].rename(columns={'avg_monthly_balance_KZT':'avg_balance'})
        features = features.merge(tmp, on='client_code', how='left')
        features['avg_balance'] = features['avg_balance'].fillna(0.0)
    else:
        features['avg_balance'] = 0.0
if 'benefit_cols' not in globals():
    benefit_cols = [c for c in features.columns if c.startswith('benefit_')]
else:
    benefit_cols = [c for c in benefit_cols if c in features.columns]

out_cols = ['client_code','name','city','avg_balance'] + benefit_cols + ['best_product','best_product_tiebreak','top1_category']

for col in out_cols:
    if col not in features.columns:
        features[col] = np.nan if col not in ['client_code'] else features.get('client_code', np.nan)

result = features[out_cols].copy()

result.to_csv(output_path, index=False)
print(f"Saved results to {output_path} ({result.shape[0]} rows)")
print("\nPreview (first 10 rows):")
print(result.head(10).to_string(index=False))

print("\nSummary of chosen best products (top counts):")
print(result['best_product'].value_counts(dropna=True).head(20))
try:
    from IPython.display import display
    display(result.head(20))
except Exception:
    pass

Saved results to E:\Alex\Github\Decentraphone\output\enefits_per_client.csv (60 rows)

Preview (first 10 rows):
 client_code    name             city  avg_balance  benefit_travel  benefit_premium  benefit_credit  benefit_fx  benefit_cashloan  benefit_saving  benefit_multidep  benefit_accumulative  benefit_invest  benefit_gold best_product best_product_tiebreak top1_category
           1 Айгерим           Алматы      92643.0       73.011434        36.179010       56.456450         0.0         54.778637       14.399130          0.680534             18.745151        0.680534     14.263023       travel                travel      card_out
           2  Данияр           Астана    1577073.0       74.724582        55.975622       39.004160         0.0         29.814118       61.209513         22.635320             66.522470       22.635320     56.682449       travel                travel      card_out
           3  Сабина           Алматы      63116.0       83.786214        27.952392       42.

,client_code,name,city,avg_balance,benefit_travel,benefit_premium,benefit_credit,benefit_fx,benefit_cashloan,benefit_saving,benefit_multidep,benefit_accumulative,benefit_invest,benefit_gold,best_product,best_product_tiebreak,top1_category
0,1,Айгерим,Алматы,92643.0,73.011434,36.179010,56.456450,0.0,54.778637,14.399130,0.680534,18.745151,0.680534,14.263023,travel,travel,card_out
1,2,Данияр,Астана,1577073.0,74.724582,55.975622,39.004160,0.0,29.814118,61.209513,22.635320,66.522470,22.635320,56.682449,travel,travel,card_out
2,3,Сабина,Алматы,63116.0,83.786214,27.952392,42.503458,85.0,63.675436,15.440268,50.243829,20.424472,30.243829,15.391503,fx,fx,card_out
3,4,Тимур,Караганда,83351.0,80.250454,34.583808,52.278288,0.0,65.292878,13.732897,0.543105,17.948459,0.543105,13.624276,travel,travel,card_out
4,5,Камилла,Алматы,1336536.0,87.741935,55.170231,34.139076,0.0,23.198980,56.095228,19.077767,62.075125,19.077767,52.279674,travel,travel,card_out
5,6,Аян,Шымкент,131929.0,88.387097,35.621646,51.210595,0.0,51.935435,20.610101,1.261576,26.639083,1.261576,20.357785,travel,travel,card_out
6,7,Руслан,Алматы,4040997.0,47.767774,60.200395,36.712304,0.0,16.366860,100.000000,50.000000,100.000000,50.000000,90.000000,saving,saving,card_out
7,8,Мадина,Астана,1058403.0,46.791772,61.525965,30.076430,0.0,31.708475,49.591824,14.964168,56.146320,14.964168,46.598990,premium,premium,card_out
8,9,Арман,Алматы,3084180.0,45.895006,60.801424,34.466941,0.0,32.161031,92.784155,44.925500,93.761873,44.925500,83.799054,accumulative,accumulative,card_out
9,10,Карина,Усть-Каменогорск,1277325.0,41.409771,60.957221,35.347259,0.0,26.816328,54.478750,18.202034,60.503644,18.202034,50.838343,premium,premium,card_out


In [112]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
output_path = r'E:\Alex\Github\Decentraphone\output\enefits_per_client.csv'
# 1. Подготовим признаки и таргет
# Используем нормализованные колонки и депозит/топ3 для обучения
feature_cols = ['n_avg_monthly_outflow','n_tx_count_out','n_travel_share','n_travel_tx_count',
                'n_avg_travel_ticket','n_premium_share','n_online_freq','n_usd_eur_share',
                'n_fx_freq','n_vol_cashflow','n_small_tx_freq','n_avg_balance','n_neg_balance_risk',
                'n_deposit_tier','n_share_in_top3']

X = features[feature_cols].fillna(0.0)

# Таргет - пока используем rule-based best_product_tiebreak (pseudo-label)
y = features['best_product_tiebreak'].fillna('travel')  # если пусто, присвоим default

# Преобразуем категории в числа
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# 2. Разделим на train/test
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# 3. Обучение LightGBM
clf = lgb.LGBMClassifier(
    objective='multiclass',
    num_class=len(le.classes_),
    learning_rate=0.05,
    n_estimators=500,
    random_state=42
)
clf.fit(X_train, y_train)

# 4. Предсказания на тесте
y_pred = clf.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"ML accuracy on pseudo-labels: {acc:.3f}")

# 5. Применяем модель на всех клиентах
features['best_product_ml'] = le.inverse_transform(clf.predict(X))

# 6. Сохраняем результат с ML выбором
ml_output_path = output_path.replace('.csv','_ml.csv')
ml_result_cols = ['client_code','name','city','best_product','best_product_tiebreak','best_product_ml']
ml_result = features[ml_result_cols].copy()
ml_result.to_csv(ml_output_path, index=False, encoding='utf-8-sig')
print(f"Saved ML-enhanced results to {ml_output_path}")
print(ml_result.head(10).to_string(index=False))


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000028 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 141
[LightGBM] [Info] Number of data points in the train set: 48, number of used features: 11
[LightGBM] [Info] Start training from score -1.568616
[LightGBM] [Info] Start training from score -1.232144
[LightGBM] [Info] Start training from score -3.871201
[LightGBM] [Info] Start training from score -3.871201
[LightGBM] [Info] Start training from score -3.178054
[LightGBM] [Info] Start training from score -2.261763
[LightGBM] [Info] Start training from score -1.163151
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posi

In [114]:
ml_result.head(30)

,client_code,name,city,best_product,best_product_tiebreak,best_product_ml
0,1,Айгерим,Алматы,travel,travel,travel
1,2,Данияр,Астана,travel,travel,travel
2,3,Сабина,Алматы,fx,fx,fx
3,4,Тимур,Караганда,travel,travel,travel
4,5,Камилла,Алматы,travel,travel,travel
5,6,Аян,Шымкент,travel,travel,travel
6,7,Руслан,Алматы,saving,saving,saving
7,8,Мадина,Астана,premium,premium,premium
8,9,Арман,Алматы,accumulative,accumulative,accumulative
9,10,Карина,Усть-Каменогорск,premium,premium,premium


In [115]:

final_df = ml_result[["client_code", "best_product_ml"]].rename(
    columns={"best_product_ml": "product"}
)


out = "E:\Alex\Github\Decentraphone\output\ialgoritm.csv"
final_df.to_csv(out, index=False)

print(f"Файл сохранён: {out}")
print(final_df.head())

Файл сохранён: E:\Alex\Github\Decentraphone\output\ialgoritm.csv
   client_code product
0            1  travel
1            2  travel
2            3      fx
3            4  travel
4            5  travel


In [125]:
out_df

,client_code,product,push_notification
0,1,Карта для путешествий,"Айгерим, в август у вас 14 поездок и 14 оплат ..."
1,2,Карта для путешествий,"Данияр, в август у вас 16 поездок и 16 оплат т..."
2,3,Обмен валют,"Сабина, вы часто платите в EUR. В приложении в..."
3,4,Карта для путешествий,"Тимур, в август у вас 17 поездок и 17 оплат та..."
4,5,Карта для путешествий,"Камилла, в август у вас 23 поездок и 23 оплат ..."
5,6,Карта для путешествий,"Аян, в август у вас 22 поездок и 22 оплат такс..."
6,7,Депозит Сберегательный,"Руслан, у вас остаются свободные средства. Сбе..."
7,8,Премиальная карта,"Мадина, вы держите стабильные остатки и часто ..."
8,9,Депозит Накопительный,"Арман, у вас периодически остаются небольшие с..."
9,10,Премиальная карта,"Карина, вы держите стабильные остатки и часто ..."


In [106]:
# Словарь для замены
product_map = {
    "travel": "Карта для путешествий",
    "premium": "Премиальная карта",
    "credit": "Кредитная карта",
    "fx": "Обмен валют",
    "cashloan": "Кредит наличными",
    "multidep": "Депозит Мультивалютный",
    "saving": "Депозит Сберегательный",
    "accumulative": "Депозит Накопительный",
    "invest": "Инвестиции",
    "gold": "Золотые слитки"
}

# Заменяем product по словарю
result["product"] = result["product"].map(product_map)

# Сохраняем в CSV
result.to_csv("personalized_push.csv", index=False, encoding="utf-8-sig")

print(result)


    client_code                 product  \
0             1   Карта для путешествий   
1             2   Карта для путешествий   
2             3             Обмен валют   
3             4   Карта для путешествий   
4             5   Карта для путешествий   
5             6   Карта для путешествий   
6             7  Депозит Сберегательный   
7             8       Премиальная карта   
8             9   Депозит Накопительный   
9            10       Премиальная карта   
10           11   Депозит Накопительный   
11           12   Карта для путешествий   
12           13        Кредит наличными   
13           14        Кредит наличными   
14           15        Кредит наличными   
15           16        Кредит наличными   
16           17        Кредит наличными   
17           18        Кредит наличными   
18           19        Кредит наличными   
19           20        Кредит наличными   
20           21   Депозит Накопительный   
21           22        Кредит наличными   
22         

In [ ]:


import os
import glob
import re
import pandas as pd
from datetime import datetime

MONTH_RU = ["январь","февраль","март","апрель","май","июнь","июль","август","сентябрь","октябрь","ноябрь","декабрь"]

PRODUCT_NAMES = {
    "travel": "Карта для путешествий",
    "premium": "Премиальная карта",
    "credit": "Кредитная карта",
    "fx": "Обмен валют",
    "cashloan": "Кредит наличными",
    "multidep": "Депозит Мультивалютный",
    "saving": "Депозит Сберегательный",
    "accumulative": "Депозит Накопительный",
    "invest": "Инвестиции",
    "gold": "Золотые слитки"
}

TEMPLATES = {
  "travel": "{name}, в {month} у вас {trips_count} поездок и {taxi_count} оплат такси. Тревел-карта вернёт часть расходов кешбэком и даст привилегии на билеты и отели — ездить выгоднее. Открыть",
  "premium": "{name}, вы держите стабильные остатки и часто тратите в ресторанах ({rest_count}). Премиальная карта даст повышенный кешбэк и бесплатные снятия — удобнее управлять деньгами. Открыть",
  "credit": "{name}, ваши топ-категории — {cat1}, {cat2}, {cat3}. Кредитная карта даёт до 10% в любимых категориях и бонусы на онлайн-сервисы — удобно покрывать покупки. Оформить карту",
  "fx": "{name}, вы часто платите в {fx_curr}. В приложении выгодный обмен и авто-покупка по целевому курсу помогут экономить на операциях в валюте. Настроить обмен",
  "cashloan": "{name}, у вас были крупные траты ({max_out}) — при необходимости доступен кредит наличными с гибкими сроками погашения. Узнать доступный лимит",
  "multidep": "{name}, у вас остаются свободные средства{free_amount_phrase}. Мультивалютный вклад позволит хранить и приумножать деньги в нужной валюте с удобным управлением. Открыть вклад",
  "saving": "{name}, у вас остаются свободные средства{free_amount_phrase}. Сберегательный вклад — простой способ копить и получать доход. Открыть вклад",
  "accumulative": "{name}, у вас периодически остаются небольшие суммы{free_amount_phrase}. Накопительный вклад автоматизирует отчисления и помогает собрать цель без усилий. Настроить вклад",
  "invest": "{name}, у вас есть свободные средства{free_amount_phrase}, и вы проявляли интерес к приумножению. Инвестиции с низким порогом и без комиссий помогут начать. Открыть счёт",
  "gold": "{name}, вы искали способы защитить сбережения. Покупка золотых слитков — надёжный инструмент для диверсификации и сохранения стоимости активов. Посмотреть предложения"
}

TRIP_KEYWORDS = ["такси", "такс", "таксi", "поезд", "авиа", "авиабил", "рейс", "билет", "транспорт", "uber"]

CATEGORY_MAP = {
    "card_out": "Оплата картой",
    "p2p_out": "Перевод физическому лицу",
    "deposit_topup_out": "Пополнение вклада",
    "atm_withdrawal": "Снятие в банкомате",
    "online": "Онлайн-покупки",
    "restaurants": "Рестораны",
    "taxi": "Такси",
    "avia": "Авиабилеты",
    "train": "Поезд",
    "bus": "Автобус",
}


def clean_amount(x):
    try:
        if pd.isna(x):
            return 0.0
        if isinstance(x, (int, float)):
            return float(x)
        s = str(x).replace("_", "").replace(" ", "")
        s = re.sub(r"[^\d\.,\-]", "", s)
        s = s.replace(",", ".")
        if s in ("", "-", "."):
            return 0.0
        return float(s)
    except Exception:
        return 0.0

def format_number(n, decimals=0):
    try:
        if decimals == 0:
            return f"{int(round(n)):,}".replace(",", " ")
        else:
            return f"{n:,.{decimals}f}".replace(",", " ").replace(".", ",")
    except Exception:
        return str(n)

def format_currency(amount, currency):
    if amount is None or pd.isna(amount) or amount == 0:
        return ""
    symbol_map = {"KZT": "₸", "USD": "$", "EUR": "€"}
    sym = symbol_map.get(currency, currency)
    if sym in ["$", "€", "₸"]:
        return f"{format_number(amount, 0)} {sym}"
    else:
        return f"{format_number(amount, 0)} {currency}"

def humanize_category(cat):
    if pd.isna(cat) or str(cat).strip() == "":
        return "прочее"
    s = str(cat).strip()
    if s in CATEGORY_MAP:
        return CATEGORY_MAP[s]
    s2 = re.sub(r"(_out|_in)$", "", s)
    s2 = s2.replace("_", " ").lower().strip()
    return s2.capitalize() if s2 else "прочее"

def enforce_text_policy(text):
    text = re.sub(r"!+", "!", text)
    text = re.sub(r"([А-ЯЁ]{2,})", lambda m: m.group(1).lower(), text)
    if len(text) > 220:
        idx = text.rfind(" ", 0, 217)
        text = (text[:idx] + "...") if idx > 0 else (text[:217] + "...")
    return text

IN_KEYS = ["in", "income", "зарплата", "salary", "transfer", "topup", "deposit", "перевод", "поступление", "пополнение"]
OUT_KEYS = ["out", "purchase", "card_out", "withdrawal", "оплата", "списание", "payment"]

def is_incoming_row(row):
    for f in ["direction", "type", "status", "category"]:
        if f in row and pd.notna(row[f]):
            v = str(row[f]).lower()
            for k in IN_KEYS:
                if k in v:
                    return True
    return False


ialg_path = r"E:\Alex\Github\Decentraphone\output\ialgoritm.csv"
clients_combined_path = r"client_n_merged.csv"
clients_glob_pattern = r"E:\Alex\Github\Decentraphone\data\processed_data\client_*_merged.csv"
clients_info_path = r"E:\Alex\Github\Decentraphone\data\case 1\clients.csv"  # здесь avg_monthly_balance_KZT
output_path = r"E:\Alex\Github\Decentraphone\Final_answer\push_output.csv"


if not os.path.exists(ialg_path):
    raise FileNotFoundError(f"Не найден ialgoritm.csv: {ialg_path}")
if not os.path.exists(clients_info_path):

    clients_info = None
else:
    clients_info = pd.read_csv(clients_info_path, dtype={"client_code": str})
    clients_info["client_code"] = clients_info["client_code"].astype(str).str.strip()

    if "avg_monthly_balance_KZT" in clients_info.columns:
        clients_info["avg_monthly_balance_KZT_clean"] = clients_info["avg_monthly_balance_KZT"].apply(clean_amount)
    else:
        clients_info["avg_monthly_balance_KZT_clean"] = 0.0


ialg = pd.read_csv(ialg_path, dtype={"client_code": str, "product": str})
ialg["client_code"] = ialg["client_code"].astype(str).str.strip()


if os.path.exists(clients_combined_path):
    clients_df = pd.read_csv(clients_combined_path, dtype={"client_code": str})
else:
    files = sorted(glob.glob(clients_glob_pattern))
    if len(files) == 0:
        raise FileNotFoundError(f"Не найдены файлы клиентов: {clients_glob_pattern} и нет {clients_combined_path}")
    parts = [pd.read_csv(f, dtype={"client_code": str}) for f in files]
    clients_df = pd.concat(parts, ignore_index=True)

clients_df["client_code"] = clients_df["client_code"].astype(str).str.strip()
clients_df["amount_clean"] = clients_df["amount"].apply(clean_amount)
clients_df["date_parsed"] = pd.to_datetime(clients_df["date"], errors="coerce")
clients_df["direction"] = clients_df["direction"].fillna("").astype(str).str.lower()
clients_df["type"] = clients_df.get("type", "").astype(str).str.lower()
clients_df["status"] = clients_df.get("status", "").astype(str).str.lower()
clients_df["category"] = clients_df["category"].fillna("").astype(str)

grouped = clients_df.groupby("client_code")

NOW = pd.Timestamp("2025-09-14")
out_rows = []

for _, r in ialg.iterrows():
    cc = str(r["client_code"]).strip()
    prod_code = str(r["product"]).strip()
    prod_full = PRODUCT_NAMES.get(prod_code, prod_code)

    if cc not in grouped.groups:
        text = f"Клиент {cc}, нет данных для персонализации. Посмотреть"
        out_rows.append({"client_code": int(cc) if cc.isdigit() else cc, "product": prod_full, "push_notification": text})
        continue

    df = grouped.get_group(cc).copy()
    name = df["name"].iloc[0] if "name" in df.columns else f"Клиент {cc}"
    last_date = df["date_parsed"].max()
    month_name = MONTH_RU[last_date.month - 1] if pd.notna(last_date) else ""
    if pd.notna(last_date):
        df_month = df[(df["date_parsed"].dt.month == last_date.month) & (df["date_parsed"].dt.year == last_date.year)]
    else:
        df_month = df

    trips_mask = df_month["category"].str.lower().apply(lambda s: any(k in s for k in TRIP_KEYWORDS))
    trips_count = int(trips_mask.sum())
    taxi_count = int(df_month["category"].str.contains("такси", case=False).sum())
    rest_count = int(df["category"].str.contains("рест", case=False).sum())

    df["is_incoming"] = df.apply(is_incoming_row, axis=1)
    out_spends = df[~df["is_incoming"]].groupby("category", dropna=False)["amount_clean"].sum().sort_values(ascending=False)
    top_cats_raw = list(out_spends.index.astype(str)[:3])
    while len(top_cats_raw) < 3:
        top_cats_raw.append("-")
    top_cats = [humanize_category(c) if c and c != "-" else "-" for c in top_cats_raw]

    currency_counts = df[~df["is_incoming"]]["currency"].fillna("KZT").value_counts()
    fx_curr = None
    for cur in currency_counts.index:
        if cur != "KZT":
            fx_curr = cur; break
    if fx_curr is None:
        fx_curr = currency_counts.index[0] if len(currency_counts) > 0 else "KZT"

    dominant_currency = df["currency"].mode().iloc[0] if not df["currency"].mode().empty else "KZT"

    free_amount_str = ""
    free_amount_phrase = ""
    if clients_info is not None:
        row_info = clients_info[clients_info["client_code"].astype(str) == cc]
        if not row_info.empty:
       
            val = float(row_info["avg_monthly_balance_KZT_clean"].iloc[0]) if "avg_monthly_balance_KZT_clean" in row_info.columns else 0.0
            if val > 0.5:
                free_amount_str = format_currency(val, "KZT")
                free_amount_phrase = f" — {free_amount_str}"

    if free_amount_phrase == "":
        cutoff = NOW - pd.Timedelta(days=90)
        df_90 = df[df["date_parsed"] >= cutoff]
        df_scope = df_90 if not df_90.empty else df.copy()

        net_by_currency = {}
        for cur, g in df_scope.groupby("currency"):
            cur_key = cur if pd.notna(cur) and cur != "" else "KZT"
            sum_in = g[g.apply(is_incoming_row, axis=1)]["amount_clean"].sum()
            sum_out = g[~g.apply(is_incoming_row, axis=1)]["amount_clean"].sum()
            net_by_currency[cur_key] = float(sum_in - sum_out)

        best_cur, best_net = None, 0.0
        for cur, net in net_by_currency.items():
            if net > best_net:
                best_net = net
                best_cur = cur

        if best_net <= 0:
            total_in = df_scope[df_scope.apply(is_incoming_row, axis=1)]["amount_clean"].sum()
            total_out = df_scope[~df_scope.apply(is_incoming_row, axis=1)]["amount_clean"].sum()
            total_net = float(total_in - total_out)
            if total_net > 0:
                free_amount_val = total_net
                dominant_currency = "KZT"
            else:
                free_amount_val = 0.0
        else:
            free_amount_val = best_net
            dominant_currency = best_cur if pd.notna(best_cur) else dominant_currency

        if free_amount_val > 0.5:
            free_amount_str = format_currency(free_amount_val, dominant_currency)
            free_amount_phrase = f" — {free_amount_str}"
        else:
            free_amount_str = ""
            free_amount_phrase = ""

    # max_out
    max_out = df[~df["is_incoming"]]["amount_clean"].max()
    max_out = 0.0 if pd.isna(max_out) else float(max_out)
    max_out_str = format_currency(max_out, dominant_currency) if max_out > 0 else format_currency(0, dominant_currency)

    slots = {
        "name": name,
        "month": month_name,
        "trips_count": format_number(trips_count, 0),
        "taxi_count": format_number(taxi_count, 0),
        "rest_count": format_number(rest_count, 0),
        "cat1": top_cats[0],
        "cat2": top_cats[1],
        "cat3": top_cats[2],
        "fx_curr": fx_curr,
        "free_amount": free_amount_str,
        "free_amount_phrase": free_amount_phrase,
        "max_out": max_out_str
    }

    template = TEMPLATES.get(prod_code)
    if template is None:
        text = f"{name}, у нас есть предложение по {prod_full}. Посмотреть"
    else:
        text = template.format(**slots)

    text = enforce_text_policy(text)
    out_rows.append({"client_code": int(cc) if cc.isdigit() else cc, "product": prod_full, "push_notification": text})

# Сохранение
out_df = pd.DataFrame(out_rows)[["client_code", "product", "push_notification"]]
output_dir = os.path.dirname(output_path)
if output_dir and not os.path.exists(output_dir):
    os.makedirs(output_dir, exist_ok=True)
out_df.to_csv(output_path, index=False, encoding="utf-8")
print(f"Saved: {output_path} ({len(out_df)} rows)")


Saved: E:\Alex\Github\Decentraphone\Final_answer\push_output.csv (60 rows)
